# Adult Classifier with HELayers (HE) - Secure Aggregation in IBM FL

## Outline:
- [Add conda environment to Jupyter Notebook](#setup)
- [Federated Learning(FL)](#intro)
- [How deos IBM FL work with HE?](#HE-FL)
- [Digit Recognition](#mnist)
- [Parties](#Parties)
    - [Party Configuration](#Party-Configuration)
    - [Party Setup](#Party-Setup)
- [Register All Parties Before Starting Training](#Register-All-Parties-Before-Starting-Training)
- [Visualize Results](#Visualize-Results)
- [Shut Down](#Shut-Down)

## Add conda environment to Jupyter Notebook <a name="setup"></a>

Please ensure that you have activated the `conda` environment following the instructions in the project README.

Once done, run the following commands in your terminal to install your conda environment into the Jupyter Notebook:

1. Once you have activated the conda environment, install the `ipykernel` package: `conda install -c anaconda ipykernel`

2. Next, install the `ipykernel` module within Jupyter Notebook: `python -m ipykernel install --user --name=<conda_env>`

3. Please install the `matplotlib` package for your conda environment. 

4. Finally, restart the jupyter notebook once done. Ensure that you are running this Notebook from `<project_path>/notebooks/`, where project_path is the directory where the IBMFL repository was cloned.

When the Notebook is up and running it may prompt you to choose the kernel. Use the drop down to choose the kernel name same as that chosen when running `conda activate <conda_env>`. If no prompt shows up, you can change the kernel by clicking _Kernel_ > _Change kernel_ > _`<conda_env>`_.

## Federated Learning (FL) <a name="intro"></a>

**Federated Learning (FL)** is a distributed machine learning process in which each participant node (or party) retains their data locally and interacts with  other participants via a learning protocol. 
One main driver behind FL is the need to not share data with others  due to privacy and confidentially concerns.
Another driver is to improve the speed of training a machine learning model by leveraging other participants' training processes.

Setting up such a federated learning system requires setting up a communication infrastructure, converting machine learning algorithms to federated settings and in some cases knowing about the intricacies of security and privacy enabling techniques such as differential privacy and multi-party computation. 

In this Notebook we use [IBM FL](https://github.com/IBM/federated-learning-lib) together with Homomorphic Encryption  to have multiple parties train a classifier to recognise handwritten digits in the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). 

For a more technical dive into IBM FL, refer the whitepaper [here](https://arxiv.org/pdf/2007.10987.pdf).

In the following cells, we set up each of the components of a Federated Learning network (See Figure below) wherein all involved parties aid in training their respective local cartpoles to arrive at the upright pendulum state. In this notebook we default to 2 parties, but depending on your resources you may use more parties.

<img style="display=block; margin:auto" src="../images/FL_Network.png" width="720"/>
<p style="text-align: center">Modified from Image Source: <a href="https://arxiv.org/pdf/2007.10987.pdf">IBM Federated Learning: An Enterprise FrameworkWhite Paper V0.1</a></p>

## How does IBM FL work with HE? <a name="HE-FL"></a>

IBM FL uses the *[Cheon-Kim-Kim-Song (CKKS) scheme](https://eprint.iacr.org/2016/421.pdf)* for Homomorphic Encryption.  HE functionalities are implemented using *[IBM HElayers software development kit (SDK)](https://github.com/IBM/helayers)*, and in particular, its *[PyHElayers](https://github.com/IBM/helayers#pyhelayers-python-package)* Python package. You can install `pyhelayers` in your conda environment by running `pip install pyhelayers`. Note that `pyhelayers` is currently supported only on Linux (x86 and IBM Z).

<img style="display=block; margin:auto" src="../images/ibmfl_helayer.png" width="512"/>
<p style="text-align: center">IBM FL library integrated with HELayers </p>

After enabling IBM FL with HE, parties do not send their model updates in plaintext. Each party sends an encrypted model update and the aggregation is performed under encryption. (See the figure below.) 


![SegmentLocal](../images/FL_FHE_v3.gif "segment")
<p style="text-align: center">FL Training with HE</a></p>

### Getting things ready
We begin by setting the number of parties that will participate in the federated learning run and splitting up the data among them.

In [1]:
import sys
party_id = 0

sys.path.append('../..')
import os
os.chdir("../..")

dataset = 'adult'

## Parties

Each party holds its own dataset that is kept to itself and used to answer queries received from the aggregator. Because each party may have stored data in different formats, FL offers an abstraction called Data Handler. This module allows for custom implementations to retrieve the data from each of the participating parties. A local training handler sits at each party to control the local training happening at the party side. 

<img style="display=block; margin:auto" src="../images/FHE_stacks.png" width="680"/>
<p style="text-align: center">Aggregator and Party side configurations</a></p>

### Party Configuration

**Note**: in a typical FL setting, the parties may have very different configurations from each other. However, in this simplified example, the config does not differ much across parties. So, we first setup the configuration common to both parties, in the next cell. We discuss the parameters that are specific to each, in subsequent cells.

### Party Setup
In the following cell, we setup configurations for parties, including network-level details, hyperparameters as well as the model specifications. Please note that if you are running this notebook in distributed environment on separate nodes then you need to split the data locally and obtain the model generated by the Aggregator.

In [2]:
ctx_file = os.path.join(os.getcwd(), 'notebooks/fhe.context')
key_file = os.path.join(os.getcwd(), 'notebooks/fhe.key')
def get_party_config(party_id):
    party_config = {
        'aggregator': {
            'ip': '127.0.0.1',
            'port': 5000
        },
        'connection': {
            'info': {
                'ip': '127.0.0.1',
                'port': 8085 + party_id
            },
            'name': 'FlaskConnection',
            'path': 'ibmfl.connection.flask_connection',
            'sync': False
        },
        'data': {
            'info': {
                'txt_file': 'examples/data/adult/random/data_party'+str(party_id)+'.csv'
            },
            'name': 'AdultSklearnDataHandler',
            'path': 'ibmfl.util.data_handlers.adult_sklearn_data_handler'
        },
        'local_training': {
            'name': 'CryptoLocalTrainingHandler',
            'path': 'ibmfl.party.training.crypto_local_training_handler',
            'info': {
                'crypto': {
                    'name': 'CryptoFHE',
                    'path': 'ibmfl.crypto.helayer.fhe',
                    'key_manager': {
                        'name': 'LocalDiskKeyManager',
                        'path': 'ibmfl.crypto.keys_mng.crypto_key_mng_dsk',
                        'key_mgr_info': {
                            'files': {
                                'context': ctx_file,
                                'key': key_file
                            }
                        }
                    }
                }
            }
        },
         'model': {
            'name': 'SklearnSGDFLModel',
            'path': 'ibmfl.model.sklearn_SGD_linear_fl_model',
            'spec': {
                'model_definition': 'examples/configs/sklearn_logclassification/model_architecture.pickle'
            }
        },
        'protocol_handler': {
            'name': 'PartyProtocolHandler',
            'path': 'ibmfl.party.party_protocol_handler'
        }
    }
    return party_config

### Running the Party

Now, we invoke the `get_party_config` function to setup party and `start()` it.

Finally, we register the party with the Aggregator.

In [3]:
from ibmfl.party.party import Party

party_config = get_party_config(party_id)
party = Party(config_dict=party_config)
party.start()
party.register_party()
party.proto_handler.is_private = False  ## allows sharing of metrics with aggregator

2022-11-18 22:28:52.704 INFO   numexpr.utils :: Note: NumExpr detected 30 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-11-18 22:28:52.705 INFO   numexpr.utils :: NumExpr defaulting to 8 threads.


2022-11-18 22:28:53,444 | 1.0.7 | INFO | ibmfl.util.config                             | Getting Aggregator details from arguments.
2022-11-18 22:28:53,686 | 1.0.7 | INFO | ibmfl.util.config                             | No metrics recorder config provided for this setup.
2022-11-18 22:28:53,836 | 1.0.7 | INFO | ibmfl.util.config                             | No metrics config provided for this setup.
2022-11-18 22:28:53,837 | 1.0.7 | INFO | ibmfl.util.config                             | No evidencia recordeer config provided for this setup.
2022-11-18 22:28:53,838 | 1.0.7 | INFO | ibmfl.util.data_handlers.adult_sklearn_data_handler | Loaded training data from examples/data/adult/random/data_party0.csv
2022-11-18 22:28:54,301 | 1.0.7 | INFO | ibmfl.connection.flask_connection             | RestSender initialized
2022-11-18 22:28:54,303 | 1.0.7 | INFO | ibmfl.crypto.crypto_library                   | Initializing a key manager
2022-11-18 22:28:54,305 | 1.0.7 | INFO | ibmfl.crypto.helay

/home/swanand.kadhe/git-repos/FFL/ibmfl/util/data_handlers/adult_sklearn_data_handler.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_dataset['sex'] = training_dataset['sex'].map({' Female': 0, ' Male': 1})
/home/swanand.kadhe/git-repos/FFL/ibmfl/util/data_handlers/adult_sklearn_data_handler.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_dataset['race'] = training_dataset['race'].map(
/home/swanand.kadhe/git-repos/FFL/ibmfl/util/data_handlers/adult_sklearn_data_handler.py:161:

2022-11-18 22:28:54,334 | 1.0.7 | INFO | ibmfl.crypto.helayer.fhe                      | No flag for privacy of fusion weights in config. Setting to default value of False.
2022-11-18 22:28:54,337 | 1.0.7 | INFO | ibmfl.connection.flask_connection             | Receiver Initialized
2022-11-18 22:28:54,338 | 1.0.7 | INFO | ibmfl.connection.flask_connection             | Initializing Flask application
2022-11-18 22:28:54,341 | 1.0.7 | INFO | ibmfl.party.party                             | Party initialization successful
2022-11-18 22:28:54,342 | 1.0.7 | INFO | ibmfl.party.party                             | Party start successful
2022-11-18 22:28:54,344 | 1.0.7 | INFO | ibmfl.party.party                             | Registering party...
2022-11-18 22:28:54,345 | 1.0.7 | INFO | werkzeug                                      |  * Running on http://127.0.0.1:8085/ (Press CTRL+C to quit)
2022-11-18 22:28:54,356 | 1.0.7 | INFO | ibmfl.party.party                             | Registration Suc

2022-11-18 22:31:40,656 | 1.0.7 | INFO | ibmfl.party.training.crypto_local_training_handler | Local training done, start to encrypt model update...
2022-11-18 22:31:40,657 | 1.0.7 | INFO | ibmfl.party.training.crypto_local_training_handler | Encrypting - <class 'ibmfl.crypto.helayer.fhe.CryptoFHE'>
2022-11-18 22:31:40,662 | 1.0.7 | INFO | ibmfl.party.training.crypto_local_training_handler | Encryption done.
2022-11-18 22:31:40,669 | 1.0.7 | INFO | ibmfl.party.training.local_training_handler   | {'score': 0.725, 'acc': 0.725, 'f1': 0.35, 'precision': 0.33, 'recall': 0.38, 'average precision': 0.25, 'roc auc': 0.59, 'negative log loss': 9.5}
2022-11-18 22:31:40,670 | 1.0.7 | INFO | ibmfl.party.party_protocol_handler            | successfully finished async request
2022-11-18 22:31:45,771 | 1.0.7 | INFO | ibmfl.connection.flask_connection             | Request received for path :7
2022-11-18 22:31:45,775 | 1.0.7 | INFO | ibmfl.party.party_protocol_handler            | received a async req

## Register All Parties Before Starting Training

Now we have started and registered this Party. Next, we will start and register rest of the parties. Once all the Parties have registered we will go back to the Aggregator's notebook to start training.

## Shut Down

Invoke the `stop()` method on each of the network participants to terminate the service.

In [ ]:
party.stop()